In [2]:
%matplotlib inline
from vega import Vega
import geopandas as gpd
import pandas as pd 
import rasterio
import os
import ipywidgets
from cartoframes import Layer, BaseMap, styling, CartoContext
cc = CartoContext(base_url='https://simbiotica.carto.com', api_key='0c09d0e4d56112e17c7fc613a54f18d37223ce7c')

# Kenia Dashboard

List of indicators:
* Wildlife Numbers
* Livestock Numbers
* Livestock Numbers
* Livestock Numbers
* Land Cover
* Land Cover
* Human Wildlife Conflict
* Human Population
* Poverty/Income Levels
* Rainfall
* Temperature
* Climate Projections
* Climate Projections
* NDVI
* Road density
* Infrastructure - current
* Infrastructure - proposed
* Protected Area Boundaries
* Conservancy Boundaries
* Surface Water
* Agricultural Productivity

In [3]:
os.listdir(('/Users/alicia/Projects/jupyter-geotools-alpine/work/'+'data/keniaDashboard/Data'))

['.DS_Store',
 'Administrative_boundaries',
 'Agriculture',
 'Agriculture.zip',
 'Climate',
 'DRSRS',
 'Forest',
 'Infrastructure',
 'Landcover',
 'Livestock',
 'Livestock.zip',
 'Population',
 'Protected_areas',
 'Socioeconomic',
 'Socioeconomic.zip',
 'Water',
 'Wildlife']

## Wildlife Numbers

### Data exploration

We have two main sources in the data shared

|property|value|
|----|-------|
|agency||
|temporal range||
|frequency||

### Configuration files to merge

The config file json configuration is a list of jsons where each json represents a file to be manage. The structure is reflected on the next example:
```json
{
"path": "<valid geojson endpoint>",
"specie":"<valid species name>",
"year":1990,
"data": None, ## empty as the geodataframe will be stored here
"columns":[], ## empty array
"columns_selection":["new_name1","new_name2"],
"columns_selec_rename": {
     "new_name1":"old_name1", 
     "new_name2":"old_name2"
    }
}

```

In [20]:
basepath='/Users/alicia/Projects/jupyter-geotools-alpine/work/'
pathD=basepath+'/data/keniaDashboard/Data/Wildlife'
filenames = [{"path": os.path.join(pathD,fn), "specie":fn.split('.')[0].replace('_numbers_','_').split('_')[1], "year":(fn.split('.')[0].replace('_numbers_','_')+'_1990').split('_')[2].rstrip('s')} for fn in next(os.walk(pathD))[2] if fn.endswith(".shp")]
print(filenames)

[{'path': '/Users/alicia/Projects/jupyter-geotools-alpine/work//data/keniaDashboard/Data/Wildlife/ke_grevys_1970s.shp', 'specie': 'grevys', 'year': '1970'}, {'path': '/Users/alicia/Projects/jupyter-geotools-alpine/work//data/keniaDashboard/Data/Wildlife/ke_grevys_1990s.shp', 'specie': 'grevys', 'year': '1990'}, {'path': '/Users/alicia/Projects/jupyter-geotools-alpine/work//data/keniaDashboard/Data/Wildlife/ke_numbers_buffalo.shp', 'specie': 'buffalo', 'year': '1990'}, {'path': '/Users/alicia/Projects/jupyter-geotools-alpine/work//data/keniaDashboard/Data/Wildlife/ke_numbers_elephant.shp', 'specie': 'elephant', 'year': '1990'}, {'path': '/Users/alicia/Projects/jupyter-geotools-alpine/work//data/keniaDashboard/Data/Wildlife/ke_numbers_giraffe.shp', 'specie': 'giraffe', 'year': '1990'}, {'path': '/Users/alicia/Projects/jupyter-geotools-alpine/work//data/keniaDashboard/Data/Wildlife/ke_numbers_wildebeest.shp', 'specie': 'wildebeest', 'year': '1990'}, {'path': '/Users/alicia/Projects/jupyte

In [32]:
for files in filenames:
    files['data']=gpd.read_file(files['path'])
    files['columns']=files['data'].columns.values
    print(files['data'].head())

   ZG70M                                      geometry
0    0.0  POINT (37.51625814736857 0.2711979404049882)
1    0.0   POINT (35.40733323412545 5.017819030140916)
2    0.0    POINT (35.40749923261389 4.99412898304757)
3    0.0   POINT (35.37112301344596 4.992462559336893)
4    0.0   POINT (35.33267819952214 4.974342210941475)
   ZG90M                                      geometry
0    0.0  POINT (37.51625814736857 0.2711979404049882)
1    0.0   POINT (35.40733323412545 5.017819030140916)
2    0.0    POINT (35.40749923261389 4.99412898304757)
3    0.0   POINT (35.37112301344596 4.992462559336893)
4    0.0   POINT (35.33267819952214 4.974342210941475)
    BF90M                                           geometry
0  883.40  POLYGON ((389999.9375 234999.96875, 389999.937...
1  618.38  POLYGON ((394999.9375 229999.96875, 394999.937...
2  868.26  POLYGON ((329999.9375 134999.96875, 329999.937...
3  747.10  POLYGON ((389999.9375 84999.984375, 389999.937...
4  378.60  POLYGON ((219999.96875 7

In [44]:
for files in filenames:
    print("{0} density in {1}: **{2}**, total population: {3}".format(files['specie'],files['year'], files['data'].mean().values[0], files['data'].sum().values[0]))
print('are we talking about density? it seems that species density is more like number of individuals per grid')

grevys density in 1970: **0.4977401448866309**, total population: 10580.96
grevys density in 1990: **0.24989745037162478**, total population: 5312.32
buffalo density in 1990: **602.4837719298245**, total population: 68683.15
elephant density in 1990: **674.8346456692914**, total population: 85704.0
giraffe density in 1990: **212.04903988183162**, total population: 143557.2
wildebeest density in 1990: **1322.0032420091325**, total population: 289518.71
zebra density in 1990: **321.8727972027972**, total population: 184111.24
wildlife density in 1970: **3.0513578680203044**, total population: 64920.689999999995
wildlife density in 1990: **1.2237770257567213**, total population: 26037.08
are we talking about density? it seems that species density is more like number of individuals per grid


## Livestock Numbers

**Data Treatment:** we will convert it into a single table that will hold the next structure:

|geometry(polygons)|species|observation_year|density (count/sqkm)|  
|------------------|-------|----------------|--------------------|  
|||||  


If the density units are the same (units/sqkm) we can compare 1990 data)

In [48]:
basepath='/Users/alicia/Projects/jupyter-geotools-alpine/work/'
pathD=basepath+'/data/keniaDashboard/Data/Livestock'
os.listdir()
filenames = [{"path": os.path.join(pathD,fn), "specie":fn.split('.')[0].replace('_numbers_','_').split('_')[1], "year":(fn.split('.')[0].replace('_numbers_','_')+'_1990').split('_')[2].rstrip('s')} for fn in next(os.walk(pathD))[2] if fn.endswith(".shp")]
print(filenames)

[{'path': '/Users/alicia/Projects/jupyter-geotools-alpine/work//data/keniaDashboard/Data/Livestock/Kenya_Livestock_1990.shp', 'specie': 'Livestock', 'year': '1990'}]


In [49]:
for files in filenames:
    files['data']=gpd.read_file(files['path'])
    files['columns']=files['data'].columns.values
    print(files['data'].head())

   LVSTCK_DEN                                           geometry
0         0.0  POLYGON ((35.41802770091034 5.016674712929801,...
1         0.0  POLYGON ((35.41802770091034 5.016674712929801,...
2         0.0  POLYGON ((35.3446224755718 4.987847421356876, ...
3         0.0  POLYGON ((35.3446224755718 4.987847421356876, ...
4         0.0  POLYGON ((35.25284797684227 4.972042912677081,...


In [50]:
for files in filenames:
    print("{0} density in {1}: **{2}**, total population: {3}".format(files['specie'],files['year'], files['data'].mean().values[0], files['data'].sum().values[0]))


Livestock density in 1990: **6.702016826471141**, total population: 142592.11


## Land Cover

**Data Treatment:** we will convert it into a single table that will hold the next structure:

|geometry(polygons)|species|observation_year|density (count/sqkm)|  
|------------------|-------|----------------|--------------------|  
|||||  


If the density units are the same (units/sqkm) we can compare 1990 data)

## Key protected ecosystems

**Data Treatment:** we will convert it into a single table that will hold the next structure:

|geometry(polygons)|species|observation_year|density (count/sqkm)|  
|------------------|-------|----------------|--------------------|  
|||||  


If the density units are the same (units/sqkm) we can compare 1990 data)